# Preprocessing

### wildfire 데이터 지역명 전처리

In [ ]:
import pandas as pd

# 기준 정의
metropolitan_cities = [
    "서울",
    "부산",
    "대구",
    "인천",
    "광주",
    "대전",
    "울산",
    "세종",
    "제주",
]
exclude_cut = ["남양주", "동두천"]

for year in range(2020, 2025):
    try:
        # 데이터 불러오기
        path = f"wildFire/wildfire_data/wildfire_{year}.csv"
        df = pd.read_csv(path, encoding="utf-8-sig")

        # 지역 처리 함수
        def resolve_region(row):
            gungu = str(row.get("locgungu", "")).strip()

            if gungu in metropolitan_cities:
                return gungu
            elif gungu in exclude_cut:
                return gungu
            elif gungu and gungu.lower() != "nan":
                # 공백이 있는 경우 첫 번째 단어만 사용
                if " " in gungu:
                    return gungu.split()[0]
                # 3글자 이상인 경우 앞 두 글자만 사용
                return gungu[:2] if len(gungu) >= 3 else gungu
            else:
                return None

        df["region"] = df.apply(resolve_region, axis=1)

        # 저장
        save_path = f"wildFire/wildfire_{year}_processed.csv"
        df.to_csv(save_path, index=False, encoding="utf-8-sig")

        print(f"✅ {year}년 처리 완료 → {save_path}")
        print(f"  고유 지역 수: {df['region'].nunique()}개")

    except Exception as e:
        print(f"❌ {year}년 처리 중 오류 발생: {e}")

### weather 데이터 지역명 전처리

In [ ]:
import os

import pandas as pd

# 전처리 규칙 정의
keep_names = ["대관령", "동두천", "백령도", "서귀포", "울릉도", "추풍령", "흑산도"]
prefix_remove = ["북강릉", "북창원", "북춘천", "북부산", "서청주"]


# 지점명 정제 함수
def clean_station_name(name):
    if isinstance(name, str) and len(name) == 3:
        if name in keep_names:
            return name
        elif name in prefix_remove:
            return name[1:]
        elif name.endswith(("군", "시")):
            return name[:-1]
    return name


# 데이터 디렉토리 경로
data_dir = "/Users/shinyeonseong/source/repos/2-1/MachineLearning1/TermProject/wildFire/weather_data"

# 2020년부터 2024년까지의 데이터 처리
for year in range(2020, 2025):
    input_file = os.path.join(data_dir, f"weather_{year}.csv")
    output_file = os.path.join(data_dir, f"weather_{year}_processed.csv")

    try:
        # CSV 파일 읽기
        print(f"\n{year}년 데이터 처리 중...")
        df = pd.read_csv(input_file)

        # 지점명 전처리 적용
        df["지점명"] = df["지점명"].apply(clean_station_name)

        # 전처리된 파일 저장
        df.to_csv(output_file, index=False)
        print(f"{year}년 데이터 처리 완료: {output_file}")

    except FileNotFoundError:
        print(f"경고: {year}년 데이터 파일을 찾을 수 없습니다.")
    except Exception as e:
        print(f"오류: {year}년 데이터 처리 중 문제가 발생했습니다: {str(e)}")

print("\n모든 데이터 처리 완료!")

### 산불 발생 횟수 wildfire_count 컬럼 처리

In [ ]:
import pandas as pd

for year in range(2020, 2025):
    print(f"=== {year}년 처리 시작 ===")
    weather_path = f"wildFire/weather_processed/weather_{year}_processed.csv"
    wildfire_path = f"wildFire/wildfire_processed/wildfire_{year}_processed.csv"
    try:
        # -------------------------
        # 1. weather 데이터 처리
        # -------------------------
        weather_df = pd.read_csv(weather_path, encoding="utf-8-sig")
        weather_df["일시"] = pd.to_datetime(weather_df["일시"])
        # 날짜는 유지하고 시간만 HH:MM 형식으로 변경
        weather_df["hour"] = weather_df["일시"].dt.strftime("%Y-%m-%d %H:%M")
        weather_df = weather_df.rename(columns={"지점명": "region"})

        for col in ["기온(°C)", "강수량(mm)", "풍속(m/s)", "습도(%)"]:
            weather_df[col] = pd.to_numeric(weather_df[col], errors="coerce")

        # -------------------------
        # 2. wildfire 데이터 처리
        # -------------------------
        wildfire_df = pd.read_csv(wildfire_path, encoding="utf-8-sig")

        # 시간 파싱 안전하게 처리
        wildfire_df["datetime"] = pd.to_datetime(
            wildfire_df["startyear"].astype(str).str.zfill(4)
            + "-"
            + wildfire_df["startmonth"].astype(str).str.zfill(2)
            + "-"
            + wildfire_df["startday"].astype(str).str.zfill(2)
            + " "
            + wildfire_df["starttime"].fillna("00:00:00").str.slice(0, 8),
            errors="coerce",
        )
        # 날짜는 유지하고 시간만 HH:MM 형식으로 변경
        wildfire_df["hour"] = (
            wildfire_df["datetime"].dt.floor("h").dt.strftime("%Y-%m-%d %H:%M")
        )

        # 지역 처리: 광역시/특별시/특별자치시/도 구분
        metropolitan_cities = [
            "광주",
            "대구",
            "부산",
            "서울",
            "세종",
            "울산",
            "인천",
        ]
        wildfire_df["region"] = wildfire_df.apply(
            lambda row: (
                row["locsi"] if row["locsi"] in metropolitan_cities else row["locgungu"]
            ),
            axis=1,
        )

        # -------------------------
        # 3. 발생 여부 매핑
        # -------------------------

        # 산불 발생 시각+지역별 카운트
        wildfire_counts = (
            wildfire_df.groupby(["region", "hour"]).size().reset_index(name="count")
        )

        # 매핑 로직 개선
        def count_wildfires(row):
            # 같은 지역의 데이터만 필터링
            region_matches = wildfire_counts[wildfire_counts["region"] == row["region"]]
            if len(region_matches) == 0:
                return 0

            # 같은 시간의 데이터만 필터링
            time_matches = region_matches[region_matches["hour"] == row["hour"]]
            return time_matches["count"].sum() if len(time_matches) > 0 else 0

        weather_df["wildfire_count"] = weather_df.apply(count_wildfires, axis=1)

        # -------------------------
        # 4. 결과 정리 및 저장
        # -------------------------
        final = weather_df[
            [
                "hour",
                "region",
                "기온(°C)",
                "강수량(mm)",
                "풍속(m/s)",
                "습도(%)",
                "wildfire_count",
            ]
        ].rename(
            columns={
                "hour": "datetime",
                "기온(°C)": "temp",
                "강수량(mm)": "rain",
                "풍속(m/s)": "wind",
                "습도(%)": "humidity",
            }
        )

        for col in ["temp", "rain", "wind", "humidity"]:
            final[col] = final[col].round(3)

        output_path = f"wildFire/final_{year}.csv"
        final.to_csv(output_path, index=False, encoding="utf-8-sig")

        print(f"✅ {year}년 최종 데이터 생성 완료! 샘플:")
        print(final.head())
        print("📊 wildfire_count 분포:")
        print(final["wildfire_count"].value_counts())
        print()

    except Exception as e:
        print(f"❌ {year}년 처리 중 오류 발생: {e}")

### indicator column 추가

In [4]:
import pandas as pd

# 처리할 파일 리스트
file_list = [
    "final_data/final_2020.csv",
    "final_data/final_2021.csv",
    "final_data/final_2022.csv",
    "final_data/final_2023.csv",
    "final_data/final_2024.csv",
]

for file_path in file_list:
    df = pd.read_csv(file_path)

    # rain 값이 결측이면 1, 아니면 rain 값 그대로 복사
    df["rain_indicator"] = df["rain"].where(df["rain"].notnull(), 1)

    # 덮어쓰기
    df.to_csv(file_path, index=False)
    print(f"{file_path} 처리 완료 (rain_indicator 컬럼 추가)")

final_data/final_2020.csv 처리 완료 (rain_indicator 컬럼 추가)
final_data/final_2021.csv 처리 완료 (rain_indicator 컬럼 추가)
final_data/final_2022.csv 처리 완료 (rain_indicator 컬럼 추가)
final_data/final_2023.csv 처리 완료 (rain_indicator 컬럼 추가)
final_data/final_2024.csv 처리 완료 (rain_indicator 컬럼 추가)


### 지역별 데이터셋 분리

In [ ]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv("final_2020-2024.csv")  # 파일명에 맞게 수정

# region 별로 그룹화하여 나눔
region_groups = {region: group_df for region, group_df in df.groupby("region")}

# 각 그룹을 CSV 파일로 저장
for region, group_df in region_groups.items():
    # 지역명을 파일명으로 사용할 수 있도록 파일명 안전하게 처리 (예: 공백 제거, 특수문자 제거 등 필요 시 추가)
    safe_region = region.replace(" ", "_")
    filename = f"{safe_region}_data.csv"
    group_df.to_csv(filename, index=False)

# 전체 몇 개의 CSV 파일로 나뉘었는지 출력
print(f"총 {len(region_groups)}개의 지역별 CSV 파일로 나누어 저장되었습니다.")

총 91개의 지역별 CSV 파일로 나누어 저장되었습니다.


### KNN(temp, humidity, wind 컬럼 결측치 처리) - (1)
#### Elbow method 사용

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

# 수치형만
numeric_df = sample_df[numeric_cols].copy()
original_nan_mask = numeric_df.isna()
mask_candidates = ~original_nan_mask
random_mask = np.random.rand(*numeric_df.shape) < 0.1
mask = random_mask & mask_candidates

# 인위적 결측 생성
masked_df = numeric_df.copy()
masked_df[mask] = np.nan

# Elbow loop
rmse_list = []
for k in range(1, 16):
    imputer = KNNImputer(n_neighbors=k)
    imputed = imputer.fit_transform(masked_df)

    # 1. flatten된 mask 위치에서의 값 추출
    true_vals = numeric_df.values[mask]  # 원래 값
    pred_vals = imputed[mask]  # 복원된 값

    # 2. 길이 확인 (디버깅용)
    print(f"k={k} | true={true_vals.shape}, pred={pred_vals.shape}")

    # 3. RMSE 계산 및 저장
    rmse = np.sqrt(mean_squared_error(true_vals, pred_vals))
    rmse_list.append(rmse)

# Plot
plt.plot(range(1, 16), rmse_list, marker="o")
plt.xlabel("n_neighbors")
plt.ylabel("RMSE")
plt.title("Elbow Method for Optimal n_neighbors")
plt.grid()
plt.show()

### KNN(temp, humidity, wind 컬럼 결측치 처리) - (2)
#### KNN imputation

In [ ]:
# 결측치 처리 방법 1. KNN Imputation
### KNN Imputation: 결측치를 가장 유사한 K개의 이웃값을 평균/가중 평균하여 채우는 방식
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# KNN 전용 데이터프레임 df_KNN 생성
df_KNN = df_data_all

# 수치형 변수만 대상으로 KNN Imputation 수행
# KNNImputer는 수치형만 처리 가능하므로 문자열/범주형 제외
num_cols = df_KNN.select_dtypes(include=[np.number]).columns
from sklearn.impute import KNNImputer

# 결과 저장용 리스트
imputed_groups = []

# 수치형 컬럼
num_cols = df_KNN.select_dtypes(include=[np.number]).columns

# 지역별로 나누어 KNN Imputer 적용
imputed_groups = []
for region_name, group in df_KNN.groupby("region"):
    group_copy = group.copy()

    # 해당 그룹 내 수치형 컬럼에 대해서만 KNN 적용
    # optimal k = 4 (elbow method 참조)
    imputer = KNNImputer(n_neighbors=4)
    group_copy[num_cols] = imputer.fit_transform(group_copy[num_cols])

    imputed_groups.append(group_copy)

# 병합
df_KNN_imputed = pd.concat(imputed_groups).sort_index()

# 결과 확인
print("✅ 지역별 KNN Imputation 완료!")
print("📦 최종 데이터 크기:", df_KNN_imputed.shape)